In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from albumentations import *
import random
import cv2
import torch
from matplotlib import pyplot as plt
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [ ]:
#show nhiều ảnh
def plot_imgs(imgs, cols=5, size=7, is_rgb=False):
    rows = len(imgs)//cols + 1
    fig = plt.figure(figsize=(cols*size, rows*size))
    for i, img in enumerate(imgs):
        fig.add_subplot(rows, cols, i+1)
        if is_rgb:
            plt.imshow(img)
        else:
            plt.imshow(img[:,:,::-1])
    plt.show()

# vẽ bounding box lên ảnh
def visualize_bbox(img, boxes, thickness=3, color=(255, 0, 0)):
    img_copy = img.copy()
    for box in boxes:
        img_copy = cv2.rectangle(
            img_copy,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            color, thickness)
    return img_copy

# load ảnh
def load_img(img_id, folder):
    img_fn = f"{folder}/{img_id}.jpg"
    img = cv2.imread(img_fn).astype(np.float32)
    img /= 255.0
    return img

def plot_imgs_and_boxes(imgs, boxes, cols=5, size=7, is_rgb=False, thickness=3, color=(255, 0, 0)):
    rows = len(imgs)//cols + 1
    fig = plt.figure(figsize=(cols*size, rows*size))
    
    for i, img in enumerate(imgs):
        
        new_img = img.copy()
        
        for box in boxes[i]['boxes']:
            new_img = cv2.rectangle(
            new_img,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            color, thickness)
        
        fig.add_subplot(rows, cols, i+1)
        if is_rgb:
            plt.imshow(new_img)
        else:
            plt.imshow(new_img[:,:,::-1])
    plt.show()

In [ ]:
def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r

def read_data_in_csv(csv_path="./wheat-dataset/train.csv"):
    df = pd.read_csv(csv_path)
    df['xmin'], df['ymin'],  df['xmax'], df['ymax'] = -1, -1, -1, -1
    df[['xmin', 'ymin', 'xmax', 'ymax']] = np.stack(df['bbox'].apply(lambda x: expand_bbox(x)))
    df.drop(columns=['bbox'], inplace=True)
    df['xmin'] = df['xmin'].astype(np.float)
    df['ymin'] = df['ymin'].astype(np.float)
    df['xmax'] = df['xmax'].astype(np.float)
    df['ymax'] = df['ymax'].astype(np.float)
    df['xmax'] = df['xmax'] + df['xmin']
    df['ymax'] = df['ymax'] + df['ymin']
    objs = []
    img_ids = set(df["image_id"])
    
    for img_id in tqdm(img_ids):
        records = df[df["image_id"] == img_id]
        boxes = records[['xmin', 'ymin', 'xmax', 'ymax']].values
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        obj = {
            "img_id": img_id,
            "boxes": boxes,
            "area":area
        }
        objs.append(obj)
    return objs



In [ ]:
def get_aug(aug):
    return Compose(aug, bbox_params=BboxParams(format='pascal_voc', min_area=0, min_visibility=0, label_fields=['labels']))


class WheatDataset(Dataset):
    def __init__(self, df, img_dir, img_size, mode='train', bbox_removal_threshold=0.25):
        self.df = df
        self.img_size = img_size
        self.img_dir = img_dir
        assert mode in  ['train', 'valid']
        self.mode = mode
        self.bbox_removal_threshold = bbox_removal_threshold
        
        self.resize_transforms = get_aug([
            Resize(height=self.img_size, width=self.img_size, interpolation=1, p=1)
        ])
        
        if self.mode == 'train':
            random.shuffle(self.df)
            
        self.transform = get_aug([
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ToGray(p=0.01),
#             GaussNoise(p=0.2),
            OneOf([
                MotionBlur(p=0.2),
                MedianBlur(blur_limit=3, p=0.1),
                Blur(blur_limit=3, p=0.1),
            ], p=0.2),
            RandomBrightnessContrast(p=0.25),
#             HueSaturationValue(p=0.25)
        ])

    def load_img(self, img_id, folder):
        img_fn = f"{folder}/{img_id}.jpg"
        img = cv2.imread(img_fn).astype(np.float32)
        img /= 255.0
        return img

    def bb_overlap(self, boxA, boxB):
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])
        interArea = max(0, xB - xA) * max(0, yB - yA)
        boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
        iou = interArea / float(boxAArea)
        return iou

    def resize_image(self, image, boxes):
        cats = np.ones(boxes.shape[0], dtype=int)
        annotations = {'image': image, 'bboxes': boxes, 'labels': cats}
        augmented = self.resize_transforms(**annotations)
        image = augmented['image']
        boxes = np.array(augmented['bboxes'])
        return image, boxes        
        
    def crop_image(self, image, boxes, xmin, ymin, xmax, ymax):

        image = image[ymin:ymax,xmin:xmax,:]
        cutout_box = [xmin, ymin, xmax, ymax]
        result_boxes = []
        for box in boxes:
            iou = self.bb_overlap(box, cutout_box)
            if iou > self.bbox_removal_threshold:
                result_boxes.append(box)
        if len(result_boxes) > 0:
            result_boxes = np.array(result_boxes, dtype=float)
            result_boxes[:,[0,2]] -= xmin
            result_boxes[:,[1,3]] -= ymin
            result_boxes[:,[0,2]] = result_boxes[:,[0,2]].clip(0, xmax-xmin)
            result_boxes[:,[1,3]] = result_boxes[:,[1,3]].clip(0, ymax-ymin)
        else:
            result_boxes = np.array([], dtype=float).reshape(0,4)
        return image, result_boxes
    
    def random_crop_resize(self, image, boxes, img_size=1024, p=0.5):
        if random.random() > p:
            new_img_size = random.randint(int(0.75*img_size), img_size)
            x = random.randint(0, img_size-new_img_size)
            y = random.randint(0, img_size-new_img_size)
            image, boxes = self.crop_image(image, boxes, x, y, x+new_img_size, y+new_img_size)
            return self.resize_image(image, boxes)
        else:
            if self.img_size != 1024:
                return self.resize_image(image, boxes)
            else:
                return image, boxes
            
    def load_cutmix_image_and_boxes(self, image_index, imsize=1024):   #custom mosaic data augmentation
        image_indexs = [*range(len(self.df))]
        image_indexs.remove(image_index)
        cutmix_image_ids = [image_index] + random.sample(image_indexs, 3)
        result_image = np.full((imsize, imsize, 3), 1, dtype='float32')
        result_boxes = []
        
        xc, yc = [int(random.uniform(imsize * 0.25, imsize * 0.75)) for _ in range(2)]
        
        for i, img_id in enumerate(cutmix_image_ids):
            image, boxes = self.load_img(self.df[img_id]['img_id'], self.img_dir), self.df[img_id]['boxes']
            if i == 0:
                image, boxes = self.crop_image(image, boxes, imsize-xc, imsize-yc, imsize, imsize)
                result_image[0:yc, 0:xc,:] = image
                result_boxes.extend(boxes)
                
            elif i == 1:
                image, boxes = self.crop_image(image, boxes, 0, imsize-yc, imsize-xc, imsize)
                result_image[0:yc, xc:imsize, :] = image
                if boxes.shape[0] > 0:
                    boxes[:,[0,2]] += xc
                result_boxes.extend(boxes)
                
            elif i == 2:
                image, boxes = self.crop_image(image, boxes, 0, 0, imsize-xc, imsize-yc)
                result_image[yc:imsize, xc:imsize, :] = image
                if boxes.shape[0] > 0:
                    boxes[:,[0,2]] += xc
                    boxes[:,[1,3]] += yc
                result_boxes.extend(boxes)
                
            else:
                image, boxes = self.crop_image(image, boxes, imsize-xc, 0, imsize, imsize-yc)
                result_image[yc:imsize, 0:xc, :] = image
                if boxes.shape[0] > 0:
                    boxes[:,[1,3]] += yc
                result_boxes.extend(boxes)
                
            del image
            del boxes
        del cutmix_image_ids
        del image_indexs
        
        if len(result_boxes) == 0:
            result_boxes = np.array([], dtype=float).reshape(0,4)
        else:
            result_boxes = np.vstack(result_boxes)
            result_boxes[:,[0,2]] = result_boxes[:,[0,2]].clip(0, imsize)
            result_boxes[:,[1,3]] = result_boxes[:,[1,3]].clip(0, imsize)
            
        return result_image, result_boxes
    
    def __getitem__(self, idx):

        image, boxes = [], []
        if self.mode == 'train':
            while(True):
                
                if random.random() > 0.5:
                    image, boxes = self.load_cutmix_image_and_boxes(idx)
                else:
                    image, boxes = self.load_img(self.df[idx]['img_id'], self.img_dir), self.df[idx]['boxes']
                    image, boxes = self.random_crop_resize(image, boxes, p=0.5)
                
                if len(boxes) > 0:
                    labels = np.ones(len(boxes), dtype=int)
                    annotations = {'image': image, 'bboxes': boxes, 'labels': labels}
                    augmented = self.transform(**annotations)
                    image = augmented['image']
                    boxes = np.array(augmented['bboxes'])
                    break
        else:
            image, boxes = self.load_img(self.df[idx]['img_id'], self.img_dir), self.df[idx]['boxes']
            

        if len(boxes) == 0:
                target = {
                    "boxes": torch.zeros((0, 4), dtype=torch.float32),
                    "labels": torch.zeros(0, dtype=torch.int64),
                    "area": torch.zeros(0, dtype=torch.float32),
                    "iscrowd": torch.zeros((0,), dtype=torch.int64)
                }
        else:
            target = {}
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
            target['boxes'] = torch.as_tensor(boxes, dtype=torch.float32)
            target['labels'] = torch.ones((len(boxes),), dtype=torch.int64)
            target['area'] = torch.as_tensor(area, dtype=torch.float32)
            target['iscrowd'] = torch.zeros((len(boxes),), dtype=torch.int64)
            
#         image = torch.as_tensor(image, dtype=torch.float32)
        image = torch.from_numpy(image).permute(2,0,1)

        return image, target

        
    def __len__(self):
        return len(self.df)
    

def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
df = read_data_in_csv('../input/global-wheat-detection/train.csv')

In [ ]:
data_set = WheatDataset(df,'../input/global-wheat-detection/train',1024)

train_loader = DataLoader(
    data_set,
    batch_size=16,
    shuffle=True,
    num_workers=2,
    collate_fn=collate_fn)

# temp = iter(train_loader)

# outputs, labels = next(temp)

# plot_imgs_and_boxes(outputs,labels, is_rgb=True)


£££££££££££££££££££££££££££££££ MODEL + MIXUP + SCHEDULE LEARING RATE £££££££££££££££££££££££££££

In [ ]:
from torchvision.models.detection import FasterRCNN

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision

In [ ]:
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau


class GradualWarmupScheduler(_LRScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        self.multiplier = multiplier
        if self.multiplier < 1.:
            raise ValueError('multiplier should be greater thant or equal to 1.')
        self.total_epoch = total_epoch
        self.after_scheduler = after_scheduler
        self.finished = False
        super(GradualWarmupScheduler, self).__init__(optimizer)

    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_last_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]

        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

    def step_ReduceLROnPlateau(self, metrics, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
        self.last_epoch = epoch if epoch != 0 else 1  # ReduceLROnPlateau is called at the end of epoch, whereas others are called at beginning
        if self.last_epoch <= self.total_epoch:
            warmup_lr = [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]
            for param_group, lr in zip(self.optimizer.param_groups, warmup_lr):
                param_group['lr'] = lr
        else:
            if epoch is None:
                self.after_scheduler.step(metrics, None)
            else:
                self.after_scheduler.step(metrics, epoch - self.total_epoch)

    def step(self, epoch=None, metrics=None):
        if type(self.after_scheduler) != ReduceLROnPlateau:
            if self.finished and self.after_scheduler:
                if epoch is None:
                    self.after_scheduler.step(None)
                else:
                    self.after_scheduler.step(epoch - self.total_epoch)
                self._last_lr = self.after_scheduler.get_last_lr()
            else:
                return super(GradualWarmupScheduler, self).step(epoch)
        else:
            self.step_ReduceLROnPlateau(metrics, epoch)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Khởi tạo model
num_classes = 2
num_epochs = 2
# iters = 1
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, progress=False)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.0005, momentum=0.9, weight_decay=0.0005)

scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs-1)
scheduler = GradualWarmupScheduler(optimizer, multiplier=10, total_epoch=1, after_scheduler=scheduler_cosine)

model.to(device)


In [ ]:
# tiến hành train model
for epoch in range(num_epochs):
    print('Epoch :', epoch)
    scheduler.step(epoch)
    model.train()
    
    for images, targets in train_loader:
        
        if random.random() > 0:
            shuffle_indices = torch.randperm(len(images))
            indices = torch.arange(len(images))
            lam = np.clip(np.random.beta(1.0, 1.0), 0.35, 0.65)
            images = list(images)

            mix_targets = []
            for i, si in zip(indices, shuffle_indices):
                images[i] = lam * images[i] + (1 - lam) * images[si]
                if i.item() == si.item():
                    target = targets[i.item()]
                else:
                    target = {
                        'boxes': torch.cat([targets[i.item()]['boxes'], targets[si.item()]['boxes']]),
                        'labels': torch.cat([targets[i.item()]['labels'], targets[si.item()]['labels']]),
                        'area': torch.cat([targets[i.item()]['area'], targets[si.item()]['area']]),
                        'iscrowd': torch.cat([targets[i.item()]['iscrowd'], targets[si.item()]['iscrowd']])
                    }

                mix_targets.append(target)
            images = tuple(images)
            targets = mix_targets

        images = list(image.cuda() for image in images)
        targets = [{k: v.cuda() for k, v in t.items()} for t in targets]     

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

############## TEST MIXUP ££££££££££££££££££££££

In [ ]:
lam = np.clip(np.random.beta(1.0, 1.0), 0.35, 0.65)
mixup_image = lam * outputs[4] + (1 - lam) * outputs[10]

In [ ]:
mixup_target = {
    'boxes': torch.cat([labels[4]['boxes'], labels[10]['boxes']]),
    'labels': torch.cat([labels[4]['labels'], labels[10]['labels']]),
    'area': torch.cat([labels[4]['area'], labels[10]['area']]),
    'iscrowd': torch.cat([labels[4]['iscrowd'], labels[10]['iscrowd']])
}

In [ ]:
shuffle_indices = torch.randperm(len(outputs))
indices = torch.arange(len(outputs))
lam = np.clip(np.random.beta(1.0, 1.0), 0.35, 0.65)
new_images = torch.tenser(lam) * outputs + (1 - lam) * outputs[shuffle_indices, :]

In [ ]:

for images, targets in train_loader:
    ### mixup
    if random.random() > 0:
        shuffle_indices = torch.randperm(len(images))
        indices = torch.arange(len(images))
        lam = np.clip(np.random.beta(1.0, 1.0), 0.35, 0.65)
        images = list(images)
        
        mix_targets = []
        for i, si in zip(indices, shuffle_indices):
            images[i] = lam * images[i] + (1 - lam) * images[si]
            if i.item() == si.item():
                target = targets[i.item()]
            else:
                target = {
                    'boxes': torch.cat([targets[i.item()]['boxes'], targets[si.item()]['boxes']]),
                    'labels': torch.cat([targets[i.item()]['labels'], targets[si.item()]['labels']]),
                    'area': torch.cat([targets[i.item()]['area'], targets[si.item()]['area']]),
                    'iscrowd': torch.cat([targets[i.item()]['iscrowd'], targets[si.item()]['iscrowd']])
                }

            mix_targets.append(target)
        images = tuple(images)
        targets = mix_targets
    

    mixup_image_box = visualize_mixup_image(images[0], targets[0]['boxes'])

    plt.imshow(mixup_image_box)
    break

In [ ]:
def visualize_mixup_image(img, boxes, thickness=3, color=(255, 0, 0)):
  
    img_copy = img.permute(1,2,0).numpy()
    for box in boxes:
        img_copy = cv2.rectangle(
            img_copy,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            color, thickness)
    return img_copy

# mixup_image_box = visualize_mixup_image(mixup_image, mixup_target['boxes'])

# plt.imshow(mixup_image_box)

In [ ]:
loop

In [ ]:
len(outputs), len(labels)

######################## MODEL ##################################

In [ ]:
from torchvision.models.detection import FasterRCNN

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Khởi tạo model
num_classes = 2
num_epochs = 10
# iters = 1
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True, progress=False)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.0005, momentum=0.9, weight_decay=0.0005)
model.to(device)



In [ ]:
# def data_to_device(images, targets, device=torch.device("cuda")):
#     images = list(image.to(device) for image in images)
#     targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
#     return images, targets

In [ ]:

# tiến hành train model
for epoch in range(num_epochs):
    print('Epoch :', epoch)
    model.train()
    for images, targets in train_loader:
        
        images = list(image.cuda() for image in images)
        targets = [{k: v.cuda() for k, v in t.items()} for t in targets]     
        
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        


In [ ]:
torch.save(model.state_dict(), './model_mixup_lr_2epoch.h5')